In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import re

In [45]:
data = np.load('PEMS04.npz')
data

In [9]:
print(data)

In [10]:
data.files

['data']

In [46]:
ett = data['data'][:,:,0]


df = pd.DataFrame(ett)

# 2. 生成从 2018-01-01 00:00 开始、间隔 5 分钟、总共 16992 个时刻的时间索引
date_index = pd.date_range(
    start='2018-01-01 00:00',
    periods=ett.shape[0],
    freq='5T'
)

# 3. 在最前面插入一列 date
df.insert(0, 'date', date_index)
df

,date,0,1,2,3,4,5,6,7,8,...,297,298,299,300,301,302,303,304,305,306
0,2018-01-01 00:00:00,62.0,56.0,90.0,32.0,19.0,68.0,24.0,24.0,31.0,...,5.0,86.0,54.0,54.0,62.0,37.0,90.0,56.0,48.0,38.0
1,2018-01-01 00:05:00,61.0,40.0,94.0,25.0,16.0,64.0,22.0,7.0,29.0,...,0.0,86.0,46.0,41.0,59.0,38.0,91.0,51.0,30.0,35.0
2,2018-01-01 00:10:00,71.0,28.0,79.0,36.0,9.0,60.0,25.0,11.0,25.0,...,0.0,67.0,59.0,58.0,59.0,34.0,56.0,67.0,31.0,45.0
3,2018-01-01 00:15:00,86.0,47.0,94.0,46.0,11.0,58.0,35.0,17.0,28.0,...,22.0,101.0,82.0,75.0,74.0,75.0,42.0,126.0,64.0,81.0
4,2018-01-01 00:20:00,103.0,36.0,154.0,68.0,9.0,56.0,56.0,37.0,36.0,...,0.0,97.0,88.0,109.0,90.0,61.0,127.0,200.0,63.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16987,2018-02-28 23:35:00,68.0,78.0,85.0,53.0,17.0,168.0,90.0,23.0,27.0,...,235.0,94.0,97.0,68.0,68.0,168.0,170.0,99.0,80.0,63.0
16988,2018-02-28 23:40:00,75.0,55.0,84.0,53.0,5.0,173.0,44.0,18.0,27.0,...,242.0,105.0,117.0,79.0,64.0,147.0,171.0,93.0,63.0,60.0
16989,2018-02-28 23:45:00,66.0,76.0,77.0,50.0,15.0,150.0,61.0,16.0,34.0,...,246.0,88.0,98.0,59.0,64.0,97.0,151.0,85.0,58.0,57.0
16990,2018-02-28 23:50:00,77.0,73.0,96.0,46.0,8.0,156.0,63.0,12.0,27.0,...,201.0,94.0,101.0,48.0,67.0,117.0,165.0,69.0,52.0,43.0


In [27]:
import chinese_calendar  as cc
from chinese_calendar  import is_workday, is_holiday

ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])

# check workday or holiday
ett['is_workday'] = ett['date'].apply(lambda x: is_workday(x) and not is_holiday(x))

# dataset separation
workdays_df = ett[ett['is_workday']]
non_workdays_df = ett[~ett['is_workday']]
workdays_df.iloc[:,:-1]
# # save data
workdays_df.iloc[:,1:-1].to_csv('workdays_pems04.csv', index=False)
non_workdays_df.iloc[:,1:-1].to_csv('non_workdays_pems04.csv', index=False)
# non_workdays_df.to_csv('non_workdays_data.csv', index=False)

In [28]:
workdays_df = pd.read_csv('workdays_pems04.csv')
workdays_df

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
0,121.0,80.0,83.0,30.0,8.0,56.0,31.0,8.0,115.0,45.0,...,0.0,196.0,76.0,108.0,64.0,46.0,94.0,63.0,44.0,35.0
1,85.0,85.0,88.0,31.0,5.0,53.0,25.0,17.0,94.0,55.0,...,14.0,145.0,50.0,113.0,56.0,49.0,78.0,61.0,43.0,35.0
2,81.0,33.0,88.0,35.0,15.0,53.0,27.0,8.0,75.0,49.0,...,40.0,157.0,55.0,99.0,49.0,40.0,94.0,52.0,36.0,29.0
3,86.0,41.0,75.0,27.0,13.0,53.0,25.0,8.0,70.0,34.0,...,10.0,140.0,82.0,77.0,61.0,41.0,72.0,61.0,20.0,34.0
4,73.0,37.0,73.0,30.0,5.0,50.0,22.0,8.0,90.0,35.0,...,20.0,153.0,64.0,92.0,46.0,39.0,32.0,62.0,38.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11227,68.0,78.0,85.0,53.0,17.0,168.0,90.0,23.0,27.0,71.0,...,235.0,94.0,97.0,68.0,68.0,168.0,170.0,99.0,80.0,63.0
11228,75.0,55.0,84.0,53.0,5.0,173.0,44.0,18.0,27.0,78.0,...,242.0,105.0,117.0,79.0,64.0,147.0,171.0,93.0,63.0,60.0
11229,66.0,76.0,77.0,50.0,15.0,150.0,61.0,16.0,34.0,102.0,...,246.0,88.0,98.0,59.0,64.0,97.0,151.0,85.0,58.0,57.0
11230,77.0,73.0,96.0,46.0,8.0,156.0,63.0,12.0,27.0,104.0,...,201.0,94.0,101.0,48.0,67.0,117.0,165.0,69.0,52.0,43.0


In [29]:
df

,date,0,1,2,3,4,5,6,7,8,...,297,298,299,300,301,302,303,304,305,306
0,2018-01-01 00:00:00,62.0,56.0,90.0,32.0,19.0,68.0,24.0,24.0,31.0,...,5.0,86.0,54.0,54.0,62.0,37.0,90.0,56.0,48.0,38.0
1,2018-01-01 00:05:00,61.0,40.0,94.0,25.0,16.0,64.0,22.0,7.0,29.0,...,0.0,86.0,46.0,41.0,59.0,38.0,91.0,51.0,30.0,35.0
2,2018-01-01 00:10:00,71.0,28.0,79.0,36.0,9.0,60.0,25.0,11.0,25.0,...,0.0,67.0,59.0,58.0,59.0,34.0,56.0,67.0,31.0,45.0
3,2018-01-01 00:15:00,86.0,47.0,94.0,46.0,11.0,58.0,35.0,17.0,28.0,...,22.0,101.0,82.0,75.0,74.0,75.0,42.0,126.0,64.0,81.0
4,2018-01-01 00:20:00,103.0,36.0,154.0,68.0,9.0,56.0,56.0,37.0,36.0,...,0.0,97.0,88.0,109.0,90.0,61.0,127.0,200.0,63.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16987,2018-02-28 23:35:00,68.0,78.0,85.0,53.0,17.0,168.0,90.0,23.0,27.0,...,235.0,94.0,97.0,68.0,68.0,168.0,170.0,99.0,80.0,63.0
16988,2018-02-28 23:40:00,75.0,55.0,84.0,53.0,5.0,173.0,44.0,18.0,27.0,...,242.0,105.0,117.0,79.0,64.0,147.0,171.0,93.0,63.0,60.0
16989,2018-02-28 23:45:00,66.0,76.0,77.0,50.0,15.0,150.0,61.0,16.0,34.0,...,246.0,88.0,98.0,59.0,64.0,97.0,151.0,85.0,58.0,57.0
16990,2018-02-28 23:50:00,77.0,73.0,96.0,46.0,8.0,156.0,63.0,12.0,27.0,...,201.0,94.0,101.0,48.0,67.0,117.0,165.0,69.0,52.0,43.0


In [36]:
# check morning peak  or evening peak
ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
filtered_data = ett.loc[mask]


filtered_data.iloc[:,1:].to_csv('morning_peak_pems04.csv', index=False)
filtered_data.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
84,67.0,62.0,105.0,52.0,22.0,417.0,22.0,19.0,30.0,48.0,...,0.0,105.0,97.0,45.0,51.0,60.0,0.0,69.0,28.0,28.0
85,72.0,83.0,103.0,36.0,26.0,431.0,26.0,21.0,29.0,47.0,...,105.0,85.0,103.0,39.0,58.0,65.0,0.0,82.0,29.0,31.0
86,75.0,49.0,103.0,43.0,24.0,457.0,20.0,17.0,32.0,40.0,...,98.0,134.0,108.0,44.0,57.0,56.0,9.0,65.0,24.0,29.0
87,76.0,63.0,74.0,34.0,14.0,471.0,29.0,32.0,40.0,63.0,...,25.0,110.0,104.0,54.0,49.0,50.0,14.0,82.0,31.0,44.0
88,67.0,54.0,93.0,42.0,39.0,478.0,24.0,13.0,38.0,53.0,...,0.0,113.0,101.0,53.0,60.0,71.0,14.0,51.0,36.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16820,252.0,484.0,455.0,264.0,204.0,395.0,107.0,298.0,247.0,260.0,...,329.0,419.0,391.0,366.0,425.0,270.0,179.0,569.0,190.0,218.0
16821,434.0,381.0,410.0,264.0,227.0,385.0,111.0,409.0,212.0,259.0,...,376.0,585.0,412.0,335.0,389.0,293.0,299.0,612.0,154.0,277.0
16822,470.0,474.0,369.0,257.0,206.0,375.0,75.0,327.0,187.0,245.0,...,370.0,520.0,427.0,329.0,427.0,254.0,185.0,599.0,209.0,265.0
16823,367.0,383.0,448.0,248.0,207.0,410.0,107.0,384.0,155.0,229.0,...,405.0,546.0,408.0,325.0,457.0,274.0,260.0,562.0,174.0,263.0


In [47]:
# check morning peak  or evening peak
ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
filtered_data = ett.loc[mask]


filtered_data.iloc[:,1:].to_csv('evening_peak_pems04.csv', index=False)

filtered_data.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
204,296.0,222.0,291.0,215.0,179.0,442.0,105.0,83.0,264.0,216.0,...,477.0,614.0,363.0,336.0,209.0,239.0,108.0,339.0,157.0,169.0
205,282.0,180.0,302.0,224.0,181.0,457.0,106.0,99.0,275.0,211.0,...,328.0,517.0,384.0,323.0,244.0,244.0,204.0,327.0,182.0,175.0
206,276.0,245.0,284.0,184.0,226.0,474.0,117.0,110.0,260.0,179.0,...,451.0,652.0,384.0,312.0,203.0,243.0,227.0,346.0,190.0,222.0
207,300.0,206.0,299.0,191.0,187.0,482.0,104.0,74.0,239.0,193.0,...,435.0,628.0,373.0,350.0,220.0,245.0,216.0,407.0,169.0,188.0
208,307.0,219.0,141.0,184.0,195.0,478.0,104.0,93.0,285.0,206.0,...,428.0,563.0,367.0,336.0,222.0,224.0,287.0,326.0,170.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16940,261.0,135.0,279.0,193.0,77.0,364.0,330.0,56.0,100.0,230.0,...,510.0,262.0,347.0,160.0,216.0,360.0,436.0,327.0,297.0,179.0
16941,299.0,170.0,256.0,185.0,58.0,358.0,341.0,62.0,88.0,236.0,...,526.0,273.0,365.0,190.0,197.0,350.0,506.0,297.0,298.0,162.0
16942,285.0,169.0,127.0,179.0,58.0,344.0,305.0,46.0,104.0,199.0,...,448.0,221.0,333.0,169.0,208.0,358.0,441.0,294.0,232.0,174.0
16943,268.0,148.0,189.0,170.0,41.0,332.0,233.0,48.0,108.0,245.0,...,469.0,232.0,361.0,161.0,183.0,345.0,503.0,274.0,257.0,171.0


In [40]:
data = np.load('PEMS08.npz')
ett = data['data'][:,:,0]
ett.shape

(17856, 170)

In [49]:
data = np.load('PEMS08.npz')
ett = data['data'][:,:,0]


df = pd.DataFrame(ett)

# 2. 生成从 2018-01-01 00:00 开始、间隔 5 分钟、总共 16992 个时刻的时间索引
date_index = pd.date_range(
    start='2016-07-01 00:00',
    periods=ett.shape[0],
    freq='5T'
)

# 3. 在最前面插入一列 date
df.insert(0, 'date', date_index)
df

,date,0,1,2,3,4,5,6,7,8,...,160,161,162,163,164,165,166,167,168,169
0,2016-07-01 00:00:00,133.0,210.0,124.0,145.0,206.0,58.0,248.0,117.0,44.0,...,168.0,168.0,13.0,61.0,102.0,133.0,38.0,74.0,94.0,6.0
1,2016-07-01 00:05:00,114.0,185.0,119.0,184.0,194.0,35.0,195.0,126.0,26.0,...,193.0,197.0,13.0,66.0,96.0,125.0,40.0,73.0,84.0,4.0
2,2016-07-01 00:10:00,140.0,171.0,107.0,146.0,162.0,106.0,205.0,120.0,55.0,...,180.0,168.0,9.0,52.0,99.0,130.0,40.0,70.0,82.0,4.0
3,2016-07-01 00:15:00,106.0,174.0,98.0,148.0,193.0,65.0,213.0,115.0,52.0,...,169.0,171.0,9.0,64.0,97.0,111.0,38.0,66.0,103.0,2.0
4,2016-07-01 00:20:00,117.0,176.0,114.0,130.0,163.0,68.0,206.0,115.0,39.0,...,186.0,166.0,7.0,88.0,87.0,127.0,35.0,60.0,86.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,2016-08-31 23:35:00,131.0,191.0,149.0,140.0,213.0,59.0,232.0,111.0,44.0,...,159.0,177.0,105.0,65.0,89.0,76.0,8.0,100.0,100.0,1.0
17852,2016-08-31 23:40:00,134.0,183.0,137.0,151.0,216.0,74.0,206.0,105.0,50.0,...,203.0,179.0,103.0,59.0,90.0,84.0,4.0,89.0,97.0,2.0
17853,2016-08-31 23:45:00,120.0,176.0,119.0,142.0,194.0,68.0,237.0,105.0,40.0,...,174.0,153.0,87.0,61.0,90.0,91.0,15.0,47.0,91.0,3.0
17854,2016-08-31 23:50:00,102.0,165.0,133.0,133.0,174.0,57.0,179.0,116.0,51.0,...,168.0,154.0,97.0,32.0,78.0,80.0,7.0,97.0,85.0,2.0


In [50]:
import chinese_calendar  as cc
from chinese_calendar  import is_workday, is_holiday

ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])

# check workday or holiday
ett['is_workday'] = ett['date'].apply(lambda x: is_workday(x) and not is_holiday(x))

# dataset separation
workdays_df = ett[ett['is_workday']]
non_workdays_df = ett[~ett['is_workday']]
workdays_df.iloc[:,:-1]
# # save data
workdays_df.iloc[:,1:-1].to_csv('workdays_pems08.csv', index=False)
non_workdays_df.iloc[:,1:-1].to_csv('non_workdays_pems08.csv', index=False)
# non_workdays_df.to_csv('non_workdays_data.csv', index=False)

In [51]:
workdays_df = pd.read_csv('workdays_pems08.csv')
workdays_df

,0,1,2,3,4,5,6,7,8,9,...,160,161,162,163,164,165,166,167,168,169
0,133.0,210.0,124.0,145.0,206.0,58.0,248.0,117.0,44.0,256.0,...,168.0,168.0,13.0,61.0,102.0,133.0,38.0,74.0,94.0,6.0
1,114.0,185.0,119.0,184.0,194.0,35.0,195.0,126.0,26.0,210.0,...,193.0,197.0,13.0,66.0,96.0,125.0,40.0,73.0,84.0,4.0
2,140.0,171.0,107.0,146.0,162.0,106.0,205.0,120.0,55.0,224.0,...,180.0,168.0,9.0,52.0,99.0,130.0,40.0,70.0,82.0,4.0
3,106.0,174.0,98.0,148.0,193.0,65.0,213.0,115.0,52.0,208.0,...,169.0,171.0,9.0,64.0,97.0,111.0,38.0,66.0,103.0,2.0
4,117.0,176.0,114.0,130.0,163.0,68.0,206.0,115.0,39.0,188.0,...,186.0,166.0,7.0,88.0,87.0,127.0,35.0,60.0,86.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12667,131.0,191.0,149.0,140.0,213.0,59.0,232.0,111.0,44.0,192.0,...,159.0,177.0,105.0,65.0,89.0,76.0,8.0,100.0,100.0,1.0
12668,134.0,183.0,137.0,151.0,216.0,74.0,206.0,105.0,50.0,187.0,...,203.0,179.0,103.0,59.0,90.0,84.0,4.0,89.0,97.0,2.0
12669,120.0,176.0,119.0,142.0,194.0,68.0,237.0,105.0,40.0,180.0,...,174.0,153.0,87.0,61.0,90.0,91.0,15.0,47.0,91.0,3.0
12670,102.0,165.0,133.0,133.0,174.0,57.0,179.0,116.0,51.0,154.0,...,168.0,154.0,97.0,32.0,78.0,80.0,7.0,97.0,85.0,2.0


In [52]:
# check morning peak  or evening peak
ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
filtered_data = ett.loc[mask]


filtered_data.iloc[:,1:].to_csv('morning_peak_pems08.csv', index=False)
filtered_data.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,160,161,162,163,164,165,166,167,168,169
84,438.0,426.0,248.0,492.0,520.0,368.0,498.0,155.0,154.0,552.0,...,552.0,563.0,42.0,308.0,390.0,191.0,121.0,262.0,390.0,27.0
85,431.0,460.0,175.0,468.0,500.0,409.0,480.0,143.0,159.0,513.0,...,592.0,549.0,36.0,322.0,390.0,214.0,124.0,278.0,403.0,30.0
86,437.0,457.0,187.0,420.0,501.0,404.0,512.0,138.0,154.0,583.0,...,576.0,551.0,40.0,286.0,406.0,221.0,129.0,284.0,421.0,36.0
87,393.0,548.0,253.0,458.0,513.0,405.0,481.0,124.0,191.0,563.0,...,598.0,559.0,62.0,339.0,395.0,226.0,132.0,299.0,431.0,40.0
88,411.0,526.0,277.0,493.0,569.0,382.0,492.0,160.0,172.0,570.0,...,604.0,569.0,52.0,268.0,417.0,224.0,131.0,262.0,424.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17684,401.0,483.0,301.0,438.0,427.0,369.0,420.0,194.0,149.0,507.0,...,458.0,427.0,34.0,266.0,385.0,171.0,78.0,308.0,385.0,31.0
17685,443.0,513.0,332.0,503.0,419.0,350.0,411.0,222.0,189.0,444.0,...,483.0,373.0,43.0,258.0,424.0,151.0,69.0,263.0,427.0,33.0
17686,403.0,483.0,355.0,497.0,467.0,378.0,353.0,222.0,137.0,490.0,...,442.0,441.0,43.0,243.0,364.0,195.0,75.0,296.0,416.0,32.0
17687,439.0,438.0,313.0,486.0,506.0,359.0,369.0,237.0,169.0,505.0,...,431.0,461.0,49.0,192.0,283.0,181.0,60.0,311.0,428.0,24.0


In [ ]:
# check morning peak  or evening peak
ett = df.copy()
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
filtered_data = ett.loc[mask]


filtered_data.iloc[:,1:].to_csv('evening_peak_pems08', index=False)

filtered_data.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,160,161,162,163,164,165,166,167,168,169
204,441.0,514.0,431.0,556.0,558.0,320.0,521.0,204.0,224.0,687.0,...,533.0,547.0,103.0,310.0,422.0,246.0,134.0,225.0,418.0,57.0
205,479.0,521.0,490.0,561.0,538.0,336.0,512.0,196.0,191.0,735.0,...,567.0,569.0,104.0,285.0,344.0,231.0,136.0,195.0,439.0,64.0
206,502.0,537.0,512.0,598.0,544.0,354.0,543.0,183.0,207.0,757.0,...,565.0,545.0,89.0,287.0,360.0,224.0,140.0,247.0,420.0,53.0
207,496.0,520.0,477.0,594.0,545.0,341.0,509.0,181.0,164.0,710.0,...,581.0,529.0,104.0,215.0,375.0,254.0,142.0,282.0,468.0,54.0
208,499.0,507.0,430.0,589.0,553.0,319.0,498.0,198.0,195.0,650.0,...,576.0,525.0,86.0,270.0,380.0,214.0,141.0,265.0,444.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17804,329.0,485.0,284.0,364.0,490.0,262.0,472.0,198.0,116.0,561.0,...,394.0,320.0,46.0,138.0,252.0,255.0,36.0,271.0,335.0,23.0
17805,322.0,505.0,288.0,308.0,494.0,232.0,506.0,191.0,121.0,544.0,...,374.0,372.0,42.0,159.0,271.0,236.0,33.0,225.0,324.0,19.0
17806,265.0,542.0,298.0,311.0,459.0,229.0,477.0,223.0,99.0,520.0,...,390.0,328.0,38.0,154.0,275.0,231.0,36.0,263.0,309.0,36.0
17807,324.0,481.0,306.0,368.0,499.0,244.0,419.0,219.0,95.0,536.0,...,416.0,313.0,49.0,145.0,264.0,213.0,27.0,237.0,290.0,35.0


In [57]:
data = pd.read_csv('evening_peak_pems08.csv',header=0)
data

,0,1,2,3,4,5,6,7,8,9,...,160,161,162,163,164,165,166,167,168,169
0,441.0,514.0,431.0,556.0,558.0,320.0,521.0,204.0,224.0,687.0,...,533.0,547.0,103.0,310.0,422.0,246.0,134.0,225.0,418.0,57.0
1,479.0,521.0,490.0,561.0,538.0,336.0,512.0,196.0,191.0,735.0,...,567.0,569.0,104.0,285.0,344.0,231.0,136.0,195.0,439.0,64.0
2,502.0,537.0,512.0,598.0,544.0,354.0,543.0,183.0,207.0,757.0,...,565.0,545.0,89.0,287.0,360.0,224.0,140.0,247.0,420.0,53.0
3,496.0,520.0,477.0,594.0,545.0,341.0,509.0,181.0,164.0,710.0,...,581.0,529.0,104.0,215.0,375.0,254.0,142.0,282.0,468.0,54.0
4,499.0,507.0,430.0,589.0,553.0,319.0,498.0,198.0,195.0,650.0,...,576.0,525.0,86.0,270.0,380.0,214.0,141.0,265.0,444.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,329.0,485.0,284.0,364.0,490.0,262.0,472.0,198.0,116.0,561.0,...,394.0,320.0,46.0,138.0,252.0,255.0,36.0,271.0,335.0,23.0
2290,322.0,505.0,288.0,308.0,494.0,232.0,506.0,191.0,121.0,544.0,...,374.0,372.0,42.0,159.0,271.0,236.0,33.0,225.0,324.0,19.0
2291,265.0,542.0,298.0,311.0,459.0,229.0,477.0,223.0,99.0,520.0,...,390.0,328.0,38.0,154.0,275.0,231.0,36.0,263.0,309.0,36.0
2292,324.0,481.0,306.0,368.0,499.0,244.0,419.0,219.0,95.0,536.0,...,416.0,313.0,49.0,145.0,264.0,213.0,27.0,237.0,290.0,35.0


In [58]:
data = pd.read_csv('evening_peak_etth.csv')
data

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 17:00:00,4.756,2.344,1.635,0.711,3.076,1.492,19.134001
1,2016-07-01 18:00:00,5.626,2.880,2.523,1.208,3.076,1.492,20.681999
2,2016-07-01 19:00:00,5.492,3.014,2.452,1.208,3.015,1.553,18.712000
3,2016-07-01 20:00:00,5.358,3.014,2.452,1.208,2.863,1.523,17.868000
4,2016-07-02 17:00:00,9.913,4.823,6.894,2.416,3.229,1.736,25.466000
...,...,...,...,...,...,...,...,...
2898,2018-06-25 19:00:00,9.511,2.813,6.183,1.315,3.320,1.097,10.552000
2899,2018-06-25 20:00:00,12.994,3.483,8.457,1.635,4.447,1.249,9.989000
2900,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
2901,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [16]:
'''
Google????  

https://finance.yahoo.com/quote/%5EVXGOG/
'''

stock = pd.read_csv('stock_data.csv')
stock



,Open,High,Low,Close,Adj_Close,Volume
0,49.676899,51.693783,47.669952,49.845802,49.845802,44994500
1,50.178635,54.187561,49.925285,53.805050,53.805050,23005800
2,55.017166,56.373344,54.172661,54.346527,54.346527,18393200
3,55.260582,55.439419,51.450363,52.096165,52.096165,15361800
4,52.140873,53.651051,51.604362,52.657513,52.657513,9257400
...,...,...,...,...,...,...
3680,1207.479980,1216.300049,1200.500000,1205.920044,1205.920044,1017800
3681,1205.939941,1215.670044,1204.130005,1215.000000,1215.000000,950000
3682,1214.989990,1216.219971,1205.030029,1207.150024,1207.150024,907200
3683,1207.890015,1208.689941,1199.859985,1203.839966,1203.839966,860200


In [17]:
stock.Open.max()

1271.0

In [1]:
import yfinance as yf

# Fetch Google stock data for the specified date range
google_stock = yf.download('GOOGL', start='2004-01-01', end='2019-12-31')

# Count the number of trading days with available data
trading_days_count = len(google_stock)
print(trading_days_count)


[*********************100%***********************]  1 of 1 completed

3868


In [7]:
import os
from scipy import io
fmri = io.loadmat('./fMRI/sim4.mat')['ts']
fmri.shape

(10000, 50)

In [9]:
fmri = io.loadmat('./fMRI/sim4.mat')
fmri

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Aug 24 15:45:42 2012',
 '__version__': '1.0',
 '__globals__': [],
 'ts': array([[-1.9256301 , -2.04509448, -1.44860076, ..., -2.55091622,
         -0.79340321, -2.35513002],
        [-0.65153503, -2.84865324, -1.95699317, ...,  0.05718938,
         -0.56402644, -0.80021807],
        [ 1.40210147, -0.74846544, -0.62276712, ...,  3.10532135,
          1.70841826,  0.36818903],
        ...,
        [-1.63657374,  1.74569851, -1.72245373, ...,  0.33893141,
         -0.32003467, -2.56861468],
        [ 0.30825418,  2.17259073,  4.28112913, ...,  0.83660549,
         -1.02419497, -3.10726055],
        [ 3.53006804,  2.97773296,  4.5670008 , ...,  2.78458588,
          0.84525203, -1.39603375]]),
 'net': array([[[-1.        ,  0.35674352,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        , -1.        ,  0.23344156, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.

In [54]:
mujoco = np.load('mujoco.npy')
print('shape ',mujoco.shape)
mujoco

shape  (100, 100, 14)


array([[[1.0730922 , 0.16625899, 0.9304985 , ..., 2.090488  ,
         0.7760713 , 1.4482802 ],
        [1.0744435 , 0.16024394, 0.9286124 , ..., 2.0915098 ,
         0.77585053, 1.4482845 ],
        [1.0757959 , 0.1541302 , 0.9267283 , ..., 2.0925322 ,
         0.7756293 , 1.4482882 ],
        ...,
        [1.0991007 , 0.09363417, 0.91043955, ..., 2.0397787 ,
         0.7477329 , 1.4280124 ],
        [1.0990517 , 0.09362797, 0.91033417, ..., 2.0401466 ,
         0.7477325 , 1.4280156 ],
        [1.0990007 , 0.0936212 , 0.91022366, ..., 2.0405228 ,
         0.74773264, 1.4280177 ]],

       [[1.1642082 , 0.5365725 , 0.8264331 , ..., 2.004578  ,
         0.76847315, 1.3978573 ],
        [1.1670685 , 0.5335985 , 0.8277466 , ..., 2.0048149 ,
         0.76841146, 1.3979182 ],
        [1.1699295 , 0.53053856, 0.82905877, ..., 2.0050516 ,
         0.7683489 , 1.3979791 ],
        ...,
        [1.3780953 , 0.03927121, 0.8802937 , ..., 1.9722334 ,
         0.7881829 , 1.5640019 ],
        [1.3